# RAW Time Traces for Qubit Readout Optimal Weights

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see [Getting Started tutorial](../../../../tutorials/sources/getting_started.ipynb) if you need to create your setup and qubits for the first time. 

You can run this notebook on real hardware in the lab. However, if you don't have the hardware at your disposal, you can also run the notebook "as is" using an emulated session (see below). 

If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

In this how-to guide, you will measure the raw time traces of the readout signal coming back to the Quantum Analyzer unit of a SHFQC or SHFQA, before demodulation with the digital oscillator signal. This can be done here for every qubit in the sample for every possible combination of the states $\ket{g}$, $\ket{e}$ and $\ket{f}$. In a future version of the guide, the raw time traces will be used to determine the optimal integration kernel for qubit readout, achieving maximal distinguishability between different qubit states. 

To distinguish between $n$ different states, a total of $n(n-1)/2$ integration weights $w_{i, j}(t)$ is required, with $i, j \in\left\{0, 1, ..., n-1\right\}$ and $j>i$. At each time $t$, the optimal integration weights can be calculated as $w_{i, j}(t) = \overline{r_i(t) - r_j(t)}$, where $r_i$ denotes the time trace with the qubit in state $i$ and the horizontal bar represents the complex conjugate. Note that in general only $n-1$ different integration weights need to be explicitly measured since the remaining ones can be obtained from the pairwise difference of the measured weights.

### Imports

You'll start by importing the time-traces experiment workflows from `laboneq_applications`, as well as `laboneq.simple` and a demo QPU and device setup to run in emulation mode.

In [ ]:
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

from laboneq_applications.experiments import time_traces
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

### QPU and Device Setup

You'll generate six qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup as shown in the [Getting Started tutorial](../../../../tutorials/sources/getting_started.ipynb).

In [ ]:
my_platform = demo_platform(6)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)
session.connect(do_emulation=True)

### Create a `FolderStore` for Saving Data

The experiment `Workflows` can automatically save the inputs and outputs of all their tasks to the folder path we specify when instantiating the `FolderStore`. Here, we choose the current working directory.

In [ ]:
# import FolderStore from the `workflow` namespace of LabOne Q, which was imported
# from `laboneq.simple`
from pathlib import Path

folder_store = workflow.logbook.FolderStore(Path.cwd())

We disable saving in this guide. To enable it, simply run `folder_store.activate()`.

In [ ]:
folder_store.deactivate()

### Running the Experiment Workflow

You'll now make the experiment workflow and run. For more details on what experiment workflows are and what tasks they execute, see the [Experiment Workflows tutorial](../../../../tutorials/sources/experiment_workflows.ipynb).

Let's first create the options class for the time-traces experiment and inspect it using the tree view of the option fields per task:

In [ ]:
options = time_traces.experiment_workflow.options()
options

Or, alternatively, using the `show_fields` function from the `workflow` namespace of LabOne Q, which was imported from `laboneq.simple`:

In [ ]:
workflow.show_fields(options)

Notice that, unless we change it:

- the experiment is run in `AcquisitionType.RAW` and `AveragingMode.CYCLIC`, using 1024 averages (`count`)
- the analysis workflow will run automatically (`do_analysis=True`)
- the figures produced by the analysis are automatically closed (`close_figures=False`)
- the qubit parameters will not be updated (`update=False`)

Here, let's disable closing the figures:

In [ ]:
options.close_figures(False)

Now we run the experiment workflow on the first two qubits in parallel, for the states $|g\rangle$, $|e\rangle$, and $|f\rangle$.

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits

exp_workflow = time_traces.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    states=["g", "e", "f"],
    options=options
)

workflow_results = exp_workflow.run()

#### Inspect the Tasks That Were Run

In [ ]:
for t in workflow_results.tasks:
    print(t)

As multiplexed readout is not supported in `AcquisitionType.RAW`, we run a separate experiment for each qubit. Hence, we have as many runs of the `create_experiment`, `compile_experiment`, and `run_experiment` tasks as the number of qubits on which the experiment was executed, which in this case is two. The acquired results from the individual runs are then combined into a single instance of `RunExerimentResults` in the task `combine_results`.

#### Inspect the Output Simulation

Let's inspect the simulated output for the compiled experiment corresponding to the first qubit:

In [ ]:
compiled_exps_q0 = [t.output for t in workflow_results.tasks["compile_experiment", :] if t.index[0] == "q0"]
for compiled_experiment in compiled_exps_q0:
    plot_simulation(compiled_experiment, start_time=0, length=10e-6, signal_names_to_show=["drive", "drive_ef", "measure", "acquire"])

#### Inspecting the Source Code of the Pulse-Sequence Creation Task

You can inspect the source code of the `create_experiment` task defined in `ramsey` to see how the experiment pulse sequence is created using quantum operations. To learn more about the latter, see the Quantum Operations tutorial (add link).

In [ ]:
time_traces.create_experiment.src

To learn more about how to work with experiment `Workflows`, check out the [Experiment Workflows tutorial](../../../../tutorials/sources/experiment_workflows.ipynb).

Here, let's briefly inspect the analysis-workflow results.

#### Analysis Results

Let's check what tasks were run as part of the analysis workflow:

In [ ]:
analysis_workflow_results = workflow_results.tasks["analysis_workflow"]
for t in analysis_workflow_results.tasks:
    print(t)

We can access the qubit parameters extracted by the analysis from the output of the analysis-workflow:

In [ ]:
from pprint import pprint

pprint(analysis_workflow_results.output)  # noqa: T203

Check out the [Experiment Workflows tutorial](../../../../tutorials/sources/experiment_workflows.ipynb) to see how you can manually update the qubit parameters to these new values, or reset them to the old ones. 

Great! You've now run your resonator-spectroscopy experiment. Check out other experiments in this manual to keep characterizing your qubits.